In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from tqdm import tqdm
from collections import defaultdict
import math
import heapq
import os
import json

In [3]:
def sample_until(xs, size, seed=0):
    r = np.random.RandomState(seed)
    res = set()
    while len(res) < size and len(xs) > 0:
        new_x = r.choice(xs)
        res.add(new_x)
        xs = xs[xs != new_x]
    return list(res)

In [4]:
def sample_until_lazy(xs, size, seed=0):
    r = np.random.RandomState(seed)
    i = 0
    res = set()
    while len(res) < size and len(xs) > 0:
        new_x = r.choice(xs)
        res.add(new_x)
        i += 1
        if i % 1000 == 0:
            xs = xs[np.isin(xs, res, invert=True)]
    return list(res)

In [5]:
DATA_DIR = "/mnt/disks/data/datasets/"

# Microsoft

In [29]:
fname = os.path.join(DATA_DIR, "msft/macrobase-randomized.tsv")

In [36]:
column_names = [
    'PipelineInfo_IngestionTime',
 'SDKVersion',
 'APIVersion',
 'DeviceHash_Id',
 'AppInfo_Language',
 'AppInfo_Version',
 'DeviceInfo_Make',
 'DeviceInfo_OsBuild',
 'DeviceInfo_OsVersion',
 'DeviceInfo_Model',
 'DeviceInfo_NetworkType',
 'DeviceInfo_NetworkProvider',
 'UserInfo_Language',
 'UserInfo_TimeZone',
 'eventpriority',
 'records_received_count',
 'records_tried_to_send_count',
 'records_sent_count',
 'olsize',
 'olsize_start',
 'olc_start',
 'ol_w',
 'olc',
 'records_dropped_count',
#  'UserHash_Id',
 'inq',
 'infl',
 'r_count',
 'PipelineInfo_ClientCountry',
 'EventInfo_InitId',
 'EventInfo_Sequence',
 'e_meth',
 'TenantId',
 'DataPackageId',
 'EventInfo_Time',
 'r_no_name',
 'r_size',
 'r_ban',
 'r_kl',
 'r_ps',
 'r_403',
 'r_inv',
 'd_assert',
 'd_bad_tenant',
 'd_disk_full',
 'd_io_fail',
 'd_bond_fail',
 'd_disk_off',
 'd_unk']
df_raw = pd.read_csv(
    fname,
    names=column_names,
    header=None,
    sep="\t",
    nrows=10_000_000
)

In [46]:
f_metrics = [
    "DeviceInfo_OsBuild",
    "DeviceInfo_NetworkProvider"
]
q_metric = "records_received_count"
dims = [
    "TenantId",
    "AppInfo_Version",
    "UserInfo_TimeZone",
    "DeviceInfo_NetworkType",
]
t_col = "PipelineInfo_IngestionTime"

In [54]:
df = df_raw[f_metrics + [q_metric] + dims]

In [55]:
df[q_metric] = df[q_metric].fillna(0)

In [56]:
for cur_f in tqdm(f_metrics + dims):
    df[cur_f] = df[cur_f].fillna("na")
    vc = df[cur_f].value_counts()
    vc_rep = dict(zip(
        vc.index, 
        range(len(vc))
    ))
    df.replace({cur_f: vc_rep}, inplace=True)

100%|██████████| 6/6 [13:19<00:00, 133.31s/it]


In [60]:
out_file = os.path.join(DATA_DIR, "msft/mb-10M.csv")
df.to_csv(out_file, index=False)

In [5]:
gs = df.groupby(["TenantId", "AppInfo_Version", "UserInfo_TimeZone", "DeviceInfo_NetworkType"])

In [61]:
x_track = sample_until(df["DeviceInfo_OsBuild"], size=200, seed=0)
df_track = pd.DataFrame(
    {"x_track": x_track}
)
tname = os.path.join(DATA_DIR, "msft/mb-10M-os-track.csv")
df_track.to_csv(tname, index=False)

In [63]:
x_track = sample_until_lazy(df["DeviceInfo_NetworkProvider"], size=200, seed=0)
df_track = pd.DataFrame(
    {"x_track": x_track}
)
tname = os.path.join(DATA_DIR, "msft/mb-10M-network-track.csv")
df_track.to_csv(tname, index=False)

In [64]:
x_to_track = np.percentile(
    df["records_received_count"].dropna(),
    q=np.arange(0,101),
)
df_track = pd.DataFrame(
    {"x_track": x_to_track}
)
tname = os.path.join(DATA_DIR, "msft/mb-10M-records-track.csv")
df_track.to_csv(tname, index=False)

# Instacart Data

In [84]:
idir = os.path.join(DATA_DIR, "instacart/instacart_2017_05_01/")
tname = os.path.join(DATA_DIR, "instacart/tracked.csv")
oname = os.path.join(DATA_DIR, "instacart/p_df.feather")

In [76]:
df_orders = pd.read_csv(
    "{}/orders.csv".format(idir)
)
df_op = pd.read_csv("{}/order_products__prior.csv".format(idir))

In [77]:
df_g = df_op[["order_id", "product_id", "reordered", "add_to_cart_order"]].merge(
    df_orders[["order_id", "order_dow", "order_hour_of_day"]], 
    how="inner", on=["order_id"]
)

In [80]:
x_track = sample_until_lazy(df_g["product_id"], size=200, seed=1)
track_df = pd.DataFrame({"f": x_track})
track_df.to_csv(tname, index=False)

In [81]:
df_g["add_to_cart_order"] = np.clip(df_g["add_to_cart_order"], a_min=0, a_max=30)

In [106]:
gs = df_g.groupby(["reordered", "order_dow", "order_hour_of_day", "add_to_cart_order"])
print(len(gs))

10080


In [83]:
df_g[["product_id", "reordered", "order_dow", "order_hour_of_day", "add_to_cart_order"]].to_feather(
    oname
)

In [46]:
df_g = df_op[["order_id", "product_id"]].merge(
    df_orders[["order_id", "order_dow", "order_hour_of_day"]], 
    how="inner", on=["order_id"]
).sort_values(
    ["order_dow", "order_hour_of_day"]
).set_index(
    ["order_dow", "order_hour_of_day"]
)

In [90]:
product_lists = []
for day_of_week in tqdm(range(7)):
    for hour_of_day in range(24):
        cur_idx = (day_of_week, hour_of_day)
        if cur_idx in df_g.index:
            current_list = df_g.loc[cur_idx]["product_id"].values
            product_lists.append(current_list)

100%|██████████| 7/7 [00:00<00:00, 42.70it/s]


In [91]:
with open("/Users/edwardgan/Documents/Projects/datasets/instacart/products_grouped.txt", "w") as f:
    for plist in product_lists:
        f.write(json.dumps(plist.tolist())+"\n")

# MSFT Data

In [65]:
column_names = [
    'PipelineInfo_IngestionTime',
 'SDKVersion',
 'APIVersion',
 'DeviceHash_Id',
 'AppInfo_Language',
 'AppInfo_Version',
 'DeviceInfo_Make',
 'DeviceInfo_OsBuild',
 'DeviceInfo_OsVersion',
 'DeviceInfo_Model',
 'DeviceInfo_NetworkType',
 'DeviceInfo_NetworkProvider',
 'UserInfo_Language',
 'UserInfo_TimeZone',
 'eventpriority',
 'records_received_count',
 'records_tried_to_send_count',
 'records_sent_count',
 'olsize',
 'olsize_start',
 'olc_start',
 'ol_w',
 'olc',
 'records_dropped_count',
#  'UserHash_Id',
 'inq',
 'infl',
 'r_count',
 'PipelineInfo_ClientCountry',
 'EventInfo_InitId',
 'EventInfo_Sequence',
 'e_meth',
 'TenantId',
 'DataPackageId',
 'EventInfo_Time',
 'r_no_name',
 'r_size',
 'r_ban',
 'r_kl',
 'r_ps',
 'r_403',
 'r_inv',
 'd_assert',
 'd_bad_tenant',
 'd_disk_full',
 'd_io_fail',
 'd_bond_fail',
 'd_disk_off',
 'd_unk']

In [125]:
fname = os.path.join(DATA_DIR, "msft/")
df = pd.read_csv(
    "/Users/edwardgan/Documents/Projects/datasets/msft/mb200k.tsv",
    sep="\t",
    names=column_names
)

In [126]:
df[q_metric] = df[q_metric].fillna(0)

In [127]:
for cur_f in tqdm(f_metrics + dims):
    df[cur_f] = df[cur_f].fillna("na")
    vc = df[cur_f].value_counts()
    vc_rep = dict(zip(
        vc.index, 
        range(len(vc))
    ))
    df.replace({cur_f: vc_rep}, inplace=True)

100%|██████████| 6/6 [00:05<00:00,  1.21it/s]


In [105]:
f_metrics = [
    "DeviceInfo_OsBuild",
    "DeviceInfo_NetworkProvider"
]
q_metric = "records_received_count"
dims = [
    "TenantId",
    "AppInfo_Version",
    "UserInfo_TimeZone",
    "DeviceInfo_NetworkType",
]

In [130]:
df = pd.read_feather("/Users/edwardgan/Documents/Projects/datasets/msft/mb200k.feather")

/anaconda3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


# Avazu Data

In [170]:
df = pd.read_csv(
    "/Users/edwardgan/Documents/Projects/datasets/avazu/all/train",
    nrows=2000000
)

In [191]:
target = "site_id"
dims = ["hour", "click", "banner_pos", "app_category", "C18", "C21"]
dims = ["hour", "click", "banner_pos", "app_category", "C18", "C21"]

In [184]:
dfh.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [192]:
np.max(dfg)

33984

In [187]:
dfh = df[df["hour"] == 14102100]
dfg = dfh.groupby(dims)["id"].count()
rr = len(dfh) / np.max(dfg)
print("ratio: {}".format(rr))
print("num groups: {}".format(len(dfg)))

ratio: 3.5018243879472695
num groups: 379


In [134]:
len(dfg)

535

In [135]:
500*240

120000

# CAIDA PCap

In [6]:
df_i = pd.read_csv(os.path.join(DATA_DIR,"caida/ip_10M.csv"))

In [9]:
df = df_i.fillna("eth")
ip_map = {}
for i,k in enumerate(df["ip.dst"].value_counts().keys()):
    ip_map[k] = i
mapped_dest_ips = df["ip.dst"].map(ip_map)
df_out = pd.DataFrame()
df_out["ip.dst"] = mapped_dest_ips

In [13]:
out_file = os.path.join(DATA_DIR,"caida/caida10M-ipdst.csv")

In [15]:
df_out.to_csv(
    out_file, 
    index=False
)

In [19]:
df_out = pd.read_csv(out_file)

In [16]:
x_track = sample_until_lazy(df_out["ip.dst"],size=200)

In [17]:
track_file = os.path.join(DATA_DIR,"caida/caida10M-ipdst-xtrack.csv")

In [19]:
pd.DataFrame({"x_track": x_track}).to_csv(track_file, index=False)

# Power

In [65]:
fname = os.path.join(DATA_DIR, "power/household_power_consumption.txt")
df = pd.read_csv(
    fname,
    sep=";",
    na_values=["?"]
)

In [68]:
oname = os.path.join(DATA_DIR, "power/power.csv")
tname = os.path.join(DATA_DIR, "power/power_tracked.csv")

In [69]:
df[["Global_active_power"]].dropna().to_csv(oname, index=False)

In [70]:
x_to_track = np.percentile(
    df["Global_active_power"].dropna(),
    q=np.arange(0,101),
)

In [71]:
pd.DataFrame({"x_track": x_to_track}).to_csv(
    tname,
    index=False
)

# Zipf

In [21]:
r = np.random.RandomState(seed=0)
total_size = 10_000_000
n_max = 1_000_000
x_stream = r.zipf(1.1, size=2*total_size)
x_stream = x_stream[x_stream < n_max][:total_size]

In [20]:
fname = os.path.join(DATA_DIR, "zipf/zipf10M.csv")
tname = os.path.join(DATA_DIR, "zipf/zipf10M-xtrack.csv")

In [22]:
pd.DataFrame({"x": x_stream}).to_csv(fname, index=False)

In [24]:
df = pd.read_csv(fname)

In [25]:
x_to_track = sample_until_lazy(df["x"], size=200, seed=1)
pd.DataFrame({"x_track": x_to_track}).to_csv(tname, index=False)